#### Dependency

In [344]:
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#### Load Approved

In [346]:
data = pd.read_excel('data/curated_claims_denial.xlsx')#'data/Claim Denial Updated V7.csv')
data.shape

(2845, 68)

In [336]:
dflist = []

#### 18	Expenses incurred prior to coverage. PROCEDURE_DATE < COVERAGE_START_DATE


Rule: 1

In [337]:
indexes = []
reason_codes = []
data['COVERAGE_START_DATE'] = data['COVERAGE_START_DATE'].astype('datetime64[ns]')
data['PROCEDURE_DATE'] = data['PROCEDURE_DATE'].astype('datetime64[ns]')

for i in range(len(data)):
    if  data.loc[i, "PROCEDURE_DATE"] < data.loc[i, "COVERAGE_START_DATE"] and data.loc[i,'REASON_CODE'] == 18.0:
        reason_codes.append({'CLAIM_ID':data.loc[i,'CLAIM_ID'],
                             'CLAIM_LINE_NUMBER': data.loc[i,'CLAIM_LINE_NUMBER'],
                             'CODE': data.loc[i,'CODE'],
                             'PROCEDURE_CODE': data.loc[i,'PROCEDURE_CODE'],
                             'CLAIM_STATUS':data.loc[i,'CLAIM_STATUS'] ,
                             'REASON_CODE':data.loc[i,'REASON_CODE']})
        indexes.append(i)
dflist.append(pd.DataFrame(reason_codes))
print(len(dflist))
pd.DataFrame(reason_codes)['REASON_CODE'].value_counts()

1


18.0    30
Name: REASON_CODE, dtype: int64

This rule applies to many other reason codes apart from 18. 

#### *10	The diagnosis is inconsistent with the patient's gender.*

Gender is constant(female). map Gender to Diagnosis Code(CODE).

Rule 2

In [338]:
filtered = data[data['REASON_CODE'] == 10.0]
codes = filtered['CODE'].unique()
reason_codes = []
for i in range(len(data)):
    if  data.loc[i, "GENDER"] == 'female' and  data.loc[i, "CODE"] in codes and data.loc[i,'REASON_CODE'] == 10.0:
        reason_codes.append({'CLAIM_ID':data.loc[i,'CLAIM_ID'],
                             'CLAIM_LINE_NUMBER': data.loc[i,'CLAIM_LINE_NUMBER'],
                             'CODE': data.loc[i,'CODE'],
                             'PROCEDURE_CODE': data.loc[i,'PROCEDURE_CODE'],
                             'CLAIM_STATUS':data.loc[i,'CLAIM_STATUS'] , 
                             'REASON_CODE':data.loc[i,'REASON_CODE'] })   
        indexes.append(i)

dflist.append(pd.DataFrame(reason_codes)) 

print(len(dflist))
pd.DataFrame(reason_codes)['REASON_CODE'].value_counts()

2


10.0    30
Name: REASON_CODE, dtype: int64

In [342]:
[code  for code in codes]

['Z794',
 'I5033',
 'E10649',
 'R4182',
 'R42',
 'R918',
 'H2513',
 'I10',
 'E7800',
 'Z888',
 'R310',
 'Z825',
 'Z90710',
 'J441',
 'D122',
 'Z87891',
 'I252',
 'D125',
 'Z8546',
 'I480',
 'G8929',
 'M19022',
 'F419',
 'Z01812',
 'R079',
 'M5126',
 'N179']

#### 19	Expenses incurred after coverage terminated. 
PROCEDURE_DATE > COVERAGE_END_DATE

In [306]:
reason_codes = []
data['COVERAGE_END_DATE'] = data['COVERAGE_END_DATE'].astype('datetime64[ns]')

for i in range(len(data)):
    if  data.loc[i, "PROCEDURE_DATE"] > data.loc[i, "COVERAGE_END_DATE"] and data.loc[i,'REASON_CODE'] == 19.0:
        reason_codes.append({'CLAIM_ID':data.loc[i,'CLAIM_ID'],
                             'CLAIM_LINE_NUMBER': data.loc[i,'CLAIM_LINE_NUMBER'],
                             'CODE': data.loc[i,'CODE'],
                             'PROCEDURE_CODE': data.loc[i,'PROCEDURE_CODE'],
                             'CLAIM_STATUS':data.loc[i,'CLAIM_STATUS'],
                             'REASON_CODE':data.loc[i,'REASON_CODE']})
        indexes.append(i)
dflist.append(pd.DataFrame(reason_codes)) 

print(len(dflist))
        
pd.DataFrame(reason_codes)['REASON_CODE'].value_counts()

3


19.0    30
Name: REASON_CODE, dtype: int64

#### 9	The diagnosis is inconsistent with the patient's age. 

In [343]:
reason_codes = []
filtered = data[data['REASON_CODE'] == 9.0]
codes = filtered['CODE'].unique()
codes

array(['R079', 'E873', 'I10', 'I25810', 'I440', 'K5900', 'S4992XA',
       'M5137', 'G459', 'R0789', 'E785', 'R928', 'S42212A', 'Z90710',
       'I482', 'I739', 'I70229', 'E8342', 'F329', 'M79642', 'E806',
       'I071', 'Z23', 'M19072', 'E875', 'I4891', 'E7800'], dtype=object)

In [307]:
reason_codes = []
filtered = data[data['REASON_CODE'] == 9.0]
codes = filtered['CODE'].unique()
for i in range(len(data)):
    if  data.loc[i, "Age"] <= 19 and  data.loc[i, "CODE"] in codes:
        reason_codes.append({'CLAIM_ID':data.loc[i,'CLAIM_ID'],
                             'CLAIM_LINE_NUMBER': data.loc[i,'CLAIM_LINE_NUMBER'],
                             'CODE': data.loc[i,'CODE'],
                             'PROCEDURE_CODE': data.loc[i,'PROCEDURE_CODE'],
                             'CLAIM_STATUS':data.loc[i,'CLAIM_STATUS'] , 
                             'REASON_CODE':data.loc[i,'REASON_CODE'] }) 
        indexes.append(i)

       
dflist.append(pd.DataFrame(reason_codes)) 
print(len(dflist))        
pd.DataFrame(reason_codes)['REASON_CODE'].value_counts()

4


9.0    30
Name: REASON_CODE, dtype: int64

In [308]:
denied = pd.concat(dflist)
denied.head()

,CLAIM_ID,CLAIM_LINE_NUMBER,CODE,PROCEDURE_CODE,CLAIM_STATUS,REASON_CODE
0,4be65c41782f3a289b0ec4e009b70359,1,D125,4A023N7,Denied,18.0
1,a8da83e5dd8526ad556b2e18d0747338,1,Z01812,0W3P8ZZ,Denied,18.0
2,278cfe7a186f31122c37f22664e8057a,3,K219,3E03317,Denied,18.0
3,000adaadd604c7876ed0a6d308a1ef77,1,I10,021309W,Denied,18.0
4,be37db326ec3e681e30e56a58638f90c,1,F29,4A023N7,Denied,18.0


In [309]:
len(denied.CLAIM_ID.unique())

120

In [310]:
denied.shape

(120, 6)

In [311]:
claim_ids = denied.CLAIM_ID.tolist()
claim_lines = denied.CLAIM_LINE_NUMBER.tolist()
codes = denied.CODE.tolist()
proc_codes = denied.PROCEDURE_CODE.tolist()


In [312]:
filtered_df = data[
    (data['CLAIM_ID'].isin(claim_ids)) &
    (data['CLAIM_LINE_NUMBER'].isin(claim_lines)) &
    (data['CODE'].isin(codes)) &
    (data['PROCEDURE_CODE'].isin(proc_codes))
]
filtered_df.shape

(315, 64)

In [313]:
filtered_df['CLAIM_STATUS'].value_counts()

Denied      220
Approved     95
Name: CLAIM_STATUS, dtype: int64

In [314]:
filtered_df[filtered_df['CLAIM_STATUS']=='Denied'].shape

(220, 64)

In [315]:
""" nw_data = pd.DataFrame(columns=data.columns)
for row in data.iterrows():
    #print(row[1].keys())
    nw_data = nw_data.append(row[1])
nw_data.shape


extracted_rows = data[data['CLAIM_ID'].isin(claim_ids)]
extracted_rows.shape

df = pd.DataFrame(columns=data.columns)
for row in data.iterrows():
    if row[1][1] in  claim_ids and row[1][2] in  claim_lines and  row[1][1] in  claim_ids and  row[1][1] in  claim_ids and :
        df.add(row[1][1])  

 """

" nw_data = pd.DataFrame(columns=data.columns)\nfor row in data.iterrows():\n    #print(row[1].keys())\n    nw_data = nw_data.append(row[1])\nnw_data.shape\n\n\nextracted_rows = data[data['CLAIM_ID'].isin(claim_ids)]\nextracted_rows.shape\n\ndf = pd.DataFrame(columns=data.columns)\nfor row in data.iterrows():\n    if row[1][1] in  claim_ids and row[1][2] in  claim_lines and  row[1][1] in  claim_ids and  row[1][1] in  claim_ids and :\n        df.add(row[1][1])  \n\n "

#### 5	The procedure code/type of bill is inconsistent with the place of service

In [316]:
""" BILL_TYPE_CODE = data['BILL_TYPE_CODE'].unique()
HCPCS_CODE = data['HCPCS_CODE'].unique()
PLACE_OF_SERVICE_CODE = data['PLACE_OF_SERVICE_CODE'].unique()


reason_codes = []
for i in range(len(data)):
    if  (data.loc[i, "BILL_TYPE_CODE"] in  BILL_TYPE_CODE or  data.loc[i, "HCPCS_CODE"] in HCPCS_CODE) and \
        (data.loc[i, "PLACE_OF_SERVICE_CODE"] in PLACE_OF_SERVICE_CODE) and data.loc[i,'REASON_CODE'] == 5.0:

        reason_codes.append({'CLAIM_ID':data.loc[i,'CLAIM_ID'],
                              'CLAIM_STATUS':data.loc[i,'CLAIM_STATUS'], 
                              'REASON_CODE':data.loc[i,'REASON_CODE'] }) 
        indexes.append(i)

       
dflist.append(pd.DataFrame(reason_codes)) 
print(len(dflist))        
pd.DataFrame(reason_codes)['REASON_CODE'].value_counts() """

###############
bill_place = []
proc_place = []
reason_codes = []
for x,y,z in zip(data['BILL_TYPE_CODE'],data['HCPCS_CODE'], data['PLACE_OF_SERVICE_CODE']):
    bill_place.append((x,z))
    proc_place.append((y,z))

for i in range(len(data)):
    b_p = (data.loc[i, "BILL_TYPE_CODE"], data.loc[i, "PLACE_OF_SERVICE_CODE"]) 
    p_p = (data.loc[i, "HCPCS_CODE"], data.loc[i, "PLACE_OF_SERVICE_CODE"]) 
    if (b_p in bill_place or p_p in proc_place):# and data.loc[i,'REASON_CODE'] == 5.0:
        reason_codes.append({'CLAIM_ID':data.loc[i,'CLAIM_ID'],
                              'CLAIM_STATUS':data.loc[i,'CLAIM_STATUS'], 
                              'REASON_CODE':data.loc[i,'REASON_CODE'] }) 
        indexes.append(i)
dflist.append(pd.DataFrame(reason_codes)) 
print(len(dflist))        
pd.DataFrame(reason_codes)['REASON_CODE'].value_counts()

5


115.0    330
29.0     321
90.0     318
32.0      34
6.0       30
13.0      30
9.0       30
10.0      30
4.0       30
19.0      30
12.0      30
7.0       30
11.0      30
16.0      30
5.0       30
18.0      30
14.0      30
26.0      29
24.0      29
117.0     29
76.0      29
31.0      29
22.0      29
27.0      29
33.0      29
20.0      29
112.0     29
23.0      29
116.0     29
111.0     29
8.0       29
74.0      29
21.0      29
118.0     28
114.0     28
Name: REASON_CODE, dtype: int64

#### 7	The procedure/revenue code is inconsistent with the patient's gender

In [317]:
rev_gen = []
pro_gen = []
reason_codes = []

for x,y,z in zip(data['REVENUE_CENTER_CODE'],data['PROCEDURE_CODE'], data['GENDER']):
    rev_gen.append((x,z))
    pro_gen.append((y,z))

for i in range(len(data)):
    r_g = (data.loc[i, "REVENUE_CENTER_CODE"], data.loc[i, "GENDER"]) 
    p_g = (data.loc[i, "PROCEDURE_CODE"], data.loc[i, "GENDER"]) 
    if (r_g in rev_gen or p_g in pro_gen):# and data.loc[i,'REASON_CODE'] == 116.0:
        reason_codes.append({'CLAIM_ID':data.loc[i,'CLAIM_ID'],
                              'CLAIM_STATUS':data.loc[i,'CLAIM_STATUS'], 
                              'REASON_CODE':data.loc[i,'REASON_CODE'] }) 
        indexes.append(i)
dflist.append(pd.DataFrame(reason_codes)) 
print(len(dflist))        
pd.DataFrame(reason_codes)['REASON_CODE'].value_counts()

6


115.0    330
29.0     321
90.0     318
32.0      34
6.0       30
13.0      30
9.0       30
10.0      30
4.0       30
19.0      30
12.0      30
7.0       30
11.0      30
16.0      30
5.0       30
18.0      30
14.0      30
26.0      29
24.0      29
117.0     29
76.0      29
31.0      29
22.0      29
27.0      29
33.0      29
20.0      29
112.0     29
23.0      29
116.0     29
111.0     29
8.0       29
74.0      29
21.0      29
118.0     28
114.0     28
Name: REASON_CODE, dtype: int64

#### 116	Revenue code and Procedure code do not match.

In [318]:
lst = []
for x,y in zip(data['REVENUE_CENTER_CODE'],data['PROCEDURE_CODE']):
    lst.append((x,y))
pairs = set(lst)

reason_codes = []
for i in range(len(data)):
    pair = (data.loc[i, "REVENUE_CENTER_CODE"], data.loc[i, "PROCEDURE_CODE"]) 
    if pair in pairs:# and data.loc[i,'REASON_CODE'] == 116.0:
        reason_codes.append({'CLAIM_ID':data.loc[i,'CLAIM_ID'],
                              'CLAIM_STATUS':data.loc[i,'CLAIM_STATUS'], 
                              'REASON_CODE':data.loc[i,'REASON_CODE'] }) 
        indexes.append(i)

dflist.append(pd.DataFrame(reason_codes)) 
print(len(dflist))        
pd.DataFrame(reason_codes)['REASON_CODE'].value_counts()

7


115.0    330
29.0     321
90.0     318
32.0      34
6.0       30
13.0      30
9.0       30
10.0      30
4.0       30
19.0      30
12.0      30
7.0       30
11.0      30
16.0      30
5.0       30
18.0      30
14.0      30
26.0      29
24.0      29
117.0     29
76.0      29
31.0      29
22.0      29
27.0      29
33.0      29
20.0      29
112.0     29
23.0      29
116.0     29
111.0     29
8.0       29
74.0      29
21.0      29
118.0     28
114.0     28
Name: REASON_CODE, dtype: int64

In [319]:
tot = 0
for x in dflist:
    tot += x.shape[0]
tot

16920

#### Approved claims

##### extract approved records

In [320]:
Approved = data[data.CLAIM_STATUS == 'Approved'].copy(deep=True)
Approved.reset_index(inplace=True, drop=True)
Approved.shape

(3687, 64)

In [321]:
#18	Expenses incurred prior to coverage. PROCEDURE_DATE < COVERAGE_START_DATE
indexes = []
for i in range(len(Approved)):
    if  Approved.loc[i, "PROCEDURE_DATE"] < Approved.loc[i, "COVERAGE_START_DATE"]:
        indexes.append(i)
Approved.drop(index=indexes, inplace=True)
Approved.reset_index(inplace=True, drop=True)
Approved.shape

(3687, 64)

In [322]:
#10	The diagnosis is inconsistent with the patient's gender.
indexes = []
for i in range(len(Approved)):
    if  Approved.loc[i, "GENDER"] == 'female' and  Approved.loc[i, "CODE"] in codes:
        #Approved.drop(index=i, inplace=True)
        indexes.append(i)
Approved.drop(index=indexes, inplace=True)
Approved.reset_index(inplace=True, drop=True)
Approved.shape

(2946, 64)

In [323]:
#19	Expenses incurred after coverage terminated. 
indexes = []
for i in range(len(Approved)):
    if  Approved.loc[i, "PROCEDURE_DATE"] > Approved.loc[i, "COVERAGE_END_DATE"]:
        #Approved.drop(index=i, inplace=True)
        indexes.append(i)
Approved.drop(index=indexes, inplace=True)
Approved.reset_index(inplace=True, drop=True)
Approved.shape

(2630, 64)

In [324]:
# 9	The diagnosis is inconsistent with the patient's age. 
indexes = []
filtered = data[data['REASON_CODE'] == 9.0]
codes = filtered['CODE'].unique()
for i in range(len(Approved)):
    if  Approved.loc[i, "Age"] == 19 and  Approved.loc[i, "CODE"] in codes:
        #Approved.drop(index=i, inplace=True)
        indexes.append(i)
Approved.drop(index=indexes, inplace=True)
Approved.reset_index(inplace=True, drop=True)
Approved.shape

(2630, 64)

In [325]:
Approved.head()

,Unnamed: 0,CLAIM_ID,CLAIM_LINE_NUMBER,CLAIM_TYPE,ENCOUNTER_ID,PATIENT_ID,Name,CLAIM_START_DATE,CLAIM_END_DATE,CLAIM_LINE_START_DATE,...,MS_DRG,PROCEDURE_DATE,Year,CLAIM_STATUS,PAYERS,DENIAL_CATEGORY,REASON_CODE,PHYSICIAN_NPI,PROCEDURE_CODE,REASON_CODE_DESCRIPTION
0,1029.0,11c5940ee272f653a256810d2db1e048,1,I,11c5940ee272f653a256810d2db1e048,10042,Dorothy Hernandez,8/18/2017,8/18/2017,8/18/2017,...,202,2017-08-17,2017,Approved,BCBS,NaN,NaN,1901949420,0RGA071,"Fusion T-lum Jt w Autol Sub, Post Appr P Col, ..."
1,1030.0,11c5940ee272f653a256810d2db1e048,2,I,11c5940ee272f653a256810d2db1e048,10042,Dorothy Hernandez,8/18/2017,8/18/2017,8/18/2017,...,202,2017-08-17,2017,Approved,Medicare,NaN,NaN,1901949420,0RGA071,"Fusion T-lum Jt w Autol Sub, Post Appr P Col, ..."
2,1031.0,11c5940ee272f653a256810d2db1e048,3,I,11c5940ee272f653a256810d2db1e048,10042,Dorothy Hernandez,8/18/2017,8/18/2017,8/18/2017,...,202,2017-08-17,2017,Approved,Medicare,NaN,NaN,1901949420,0RGA071,"Fusion T-lum Jt w Autol Sub, Post Appr P Col, ..."
3,1032.0,11c5940ee272f653a256810d2db1e048,4,I,11c5940ee272f653a256810d2db1e048,10042,Dorothy Hernandez,8/18/2017,8/18/2017,8/18/2017,...,202,2017-08-17,2017,Approved,Aetna,NaN,NaN,1901949420,0RGA071,"Fusion T-lum Jt w Autol Sub, Post Appr P Col, ..."
4,1033.0,fbadd1baa2ac6e5dcf2865bf0366506c,1,I,fbadd1baa2ac6e5dcf2865bf0366506c,10556,Linda Smith,6/29/2017,6/29/2017,6/29/2017,...,378,2017-06-28,2017,Approved,Aetna,NaN,NaN,1698463008,0W3P8ZZ,"Control Bleeding in Gastrointestinal Tract, Endo"


In [326]:
merged = pd.concat([filtered_df, Approved], axis=0)
merged.shape

(2945, 64)

In [327]:
reason_codes_filtered = [18.0, 10.0, 9.0, 19.0, 1.0]

In [328]:
merged['REASON_CODE'].fillna(1.0, inplace=True)
merged.REASON_CODE.unique()

array([ 18.,  19.,  10.,   9.,  29.,   1.,  90., 115.])

In [329]:
merged = merged[merged['REASON_CODE'].isin(reason_codes_filtered)]
merged.shape

(2845, 64)

In [330]:
merged.to_csv('curated_claims_denial.csv', index=False)

In [331]:
merged.shape

(2845, 64)

In [51]:
# 116	Revenue code and Procedure code do not match.
lst = []
indexes = []
for x,y in zip(Approved['REVENUE_CENTER_CODE'],Approved['PROCEDURE_CODE']):
    lst.append((x,y))
pairs = set(lst)

for i in range(len(Approved)):
    pair = (Approved.loc[i, "REVENUE_CENTER_CODE"], Approved.loc[i, "PROCEDURE_CODE"]) 
    if pair in pairs:
        #Approved.drop(index=i, inplace=True)
        indexes.append(i)

Approved.drop(index=indexes, inplace=True)
Approved.reset_index(inplace=True, drop=True)
Approved.shape

999

In [ ]:
""" indexes = []
for i in range(len(Approved)):
    if  Approved.loc[i, "Age"] == 19 and  Approved.loc[i, "CODE"] in codes:
        Approved.drop(index=i, inplace=True)
        indexes.append(i)
Approved.drop(index=indexes, inplace=True)
Approved.reset_index(inplace=True, drop=True)
Approved.shape



#5
indexes = []
bill_place = []
proc_place = []
reason_codes = []
for x,y,z in zip(Approved['BILL_TYPE_CODE'],Approved['HCPCS_CODE'], Approved['PLACE_OF_SERVICE_CODE']):
    bill_place.append((x,z))
    proc_place.append((y,z))

for i in range(len(Approved)):
    b_p = (Approved.loc[i, "BILL_TYPE_CODE"], Approved.loc[i, "PLACE_OF_SERVICE_CODE"]) 
    p_p = (Approved.loc[i, "HCPCS_CODE"], Approved.loc[i, "PLACE_OF_SERVICE_CODE"]) 
    if (b_p in bill_place or p_p in proc_place):
        indexes.append(i)
Approved.drop(index=indexes, inplace=True)
Approved.reset_index(inplace=True, drop=True)
Approved.shape


#7
indexes = []
rev_gen = []
pro_gen = []
reason_codes = []

for x,y,z in zip(Approved['REVENUE_CENTER_CODE'],Approved['PROCEDURE_CODE'], Approved['GENDER']):
    rev_gen.append((x,z))
    pro_gen.append((y,z))

for i in range(len(Approved)):
    r_g = (Approved.loc[i, "REVENUE_CENTER_CODE"], Approved.loc[i, "GENDER"]) 
    p_g = (Approved.loc[i, "PROCEDURE_CODE"], Approved.loc[i, "GENDER"]) 
    if (r_g in rev_gen or p_g in pro_gen):
        #Approved.drop(index=i, inplace=True)
        indexes.append(i)

Approved.drop(index=indexes, inplace=True)
Approved.reset_index(inplace=True, drop=True)
Approved.head() """



""" #rule 18
reason_codes = []
dflist_app = []
indexes = []
for i in range(len(Approved)):
    if  Approved.loc[i, "PROCEDURE_DATE"] < Approved.loc[i, "COVERAGE_START_DATE"]:
        indexes.append(i)
Approved.drop(index=indexes)
Approved.reset_index(inplace=True)
Approved.shape


#rule 10
indexes = []
filtered = data[data['REASON_CODE'] == 10.0]
codes = filtered['CODE'].unique()
for i in range(len(Approved)):
    if  Approved.loc[i, "GENDER"] == 'female' and  Approved.loc[i, "CODE"] in codes:
        indexes.append(i)
Approved.drop(i, inplace=True)
Approved.reset_index(inplace=True)
Approved.shape


#rule 19
for i in range(len(Approved)):
    if  Approved.loc[i, "PROCEDURE_DATE"] > Approved.loc[i, "COVERAGE_END_DATE"]:
        Approved.drop(i, inplace=True)
#Approved.reset_index(inplace=True)
Approved.shape


#rule 9

filtered = Approved[Approved['REASON_CODE'] == 9.0]
codes = filtered['CODE'].unique()
for i in range(len(Approved)):
    if  Approved.loc[i, "Age"] == 19 and  Approved.loc[i, "CODE"] in codes:
        Approved.drop(i, inplace=True)
Approved.reset_index(inplace=True)
Approved.shape """